# Pytorch ModelBuilder

This notebook was tested with the `conda_pytorch_p310` kernel on an Amazon SageMaker notebook instance of type `m5`.

In [ ]:
!pip install boto3 sagemaker -U -q

In [ ]:
!pip install torch==2.0.1 torchvision==0.15.2 transformers==4.31.0 -q

# SageMaker ModelBuilder experience

In the new experience, we have introduced a few new constructs. Here we will focus on the following: 

1. ModelBuilder
2. SchemaBuilder
3. InferenceSpec

In the following section, we will define these constructs and provide examples to elaborate on each one.

4.1 ModelBuilder:

ModelBuilder is a Python class that takes a framework model (such as XGBoost or PyTorch) or an Inference Spec (more details below) and converts them into a SageMaker deployable model. ModelBuilder provides a `build` function that generates the artifacts for deployment. The model artifact generated is specific to the model server, which is also customizable as one of the inputs.

```python
Class definition:

class ModelBuilder(
    model_path: str | None = '/tmp/sagemaker/model-builder/' + uuid.uuid1().hex,
    role_arn: str | None = None,
    sagemaker_session: Session | None = None,
    name: str | None = 'model-name-' + uuid.uuid1().hex,
    mode: Mode | None = Mode.SAGEMAKER_ENDPOINT,
    shared_libs: List[str] = lambda : [],
    dependencies: Dict[str, Any] | None = lambda : { "auto": False },
    env_vars: Dict[str, str] | None = lambda : {},
    log_level: int | None = logging.DEBUG,
    content_type: str | None = None,
    accept_type: str | None = None,
    s3_model_data_url: str | None = None,
    instance_type: str | None = "ml.c5.xlarge",
    schema_builder: str | None = None,
    model: Any | None = None,
    inference_spec: InferenceSpec = None,
    image_uri: str | None = None,
    model_server: str | None = None
)
```
Example:

The above class file provide all the options for customization. However to deploy the framework model, the model builder just expects model, input, output and the role. 

```python
model_builder = ModelBuilder(
    model=model,  # Pass in the actual model object. It's "predict" method will be invoked in the endpoint.
    schema_builder=SchemaBuilder(input, output), # Pass in a "SchemaBuilder" which will use the sample test input and output objects to infer the serialization needed.
    role_arn=role, # Pass in the role arn or update intelligent defaults.
    )
```

4.2 SchemaBuilder:

The SchemaBuilder enables you to define the input and output for your endpoint. It allows the SchemaBuilder to generate the corresponding marshalling functions for serializing and deserializing the input and output. For further details, please consult the notebook or refer to the video.

Class definition:
```python
class SchemaBuilder(
    sample_input: Any,
    sample_output: Any,
    input_translator: CustomPayloadTranslator = None,
    output_translator: CustomPayloadTranslator = None
)
```
Example:

The CustomPayloadTranslator class provides all the options for customization. However, for [common inference data format](https://docs.aws.amazon.com/sagemaker/latest/dg/cdf-inference.html), you can just provide the sample input/output for the SchemaBuilder.
```python
input = "How is the demo going?"
output = "Comment la démo va-t-elle?"
schema = SchemaBuilder(input, output)
```

4.3 InferenceSpec

In the case you want to specify custom function to load and invoke the model instead of the framework model function, then you can pass the inference spec with your implementation in `load` and `invoke` function. 

class definition:
```python
class InferenceSpec(abc.ABC):
    @abc.abstractmethod
    def load(self, model_dir: str):
        pass

    @abc.abstractmethod
    def invoke(self, input_object: object, model: object):
        pass
```
Example:
```python
class MyInferenceSpec(InferenceSpec):
    def load(self, model_dir: str):
        return pipeline("translation_en_to_fr", model="t5-small")
        
    def invoke(self, input, model):
        return model(input)
   
inf_spec = MyInferenceSpec()

```

In this example, we are using ModelBuilder to deploy a pretrained model from Hugging Face directly. You can use `Mode` to switch between local testing and deploying to a SageMaker Endpoint. 

In [ ]:
from sagemaker import get_execution_role, Session, image_uris
import boto3
sagemaker_session = Session()
region = boto3.Session().region_name

# get execution role
# please use execution role if you are using notebook instance or update the role arn if you are using a different role
execution_role = get_execution_role() if get_execution_role() is not None else "your-role-arn"

In [ ]:
from sagemaker import image_uris
instance_type = 'ml.g5.2xlarge'
image = image_uris.retrieve(region=region, framework='huggingface', image_scope='inference', version='4.28.1', base_framework_version='pytorch2.0.0', instance_type=instance_type)

In [ ]:
from sagemaker.serve.spec.inference_spec import InferenceSpec
from transformers import pipeline

# custom inference spec with hugging face pipeline
class MyInferenceSpec(InferenceSpec):
    def load(self, model_dir: str):
        return pipeline("translation_en_to_fr", model="t5-small")
        
    def invoke(self, input, model):
        return model(input)
    
    
inf_spec = MyInferenceSpec()

In [ ]:
from sagemaker.serve.builder.model_builder import ModelBuilder
from sagemaker.serve import ModelServer
from sagemaker.serve import Mode
from sagemaker.serve.builder.schema_builder import SchemaBuilder

value: str = "Girafatron is obsessed with giraffes, the most glorious animal on the face of this Earth. Giraftron believes all other animals are irrelevant when compared to the glorious majesty of the giraffe.\nDaniel: Hello, Girafatron!\nGirafatron:"
schema = SchemaBuilder(value,
            {"generated_text": "Girafatron is obsessed with giraffes, the most glorious animal on the face of this Earth. Giraftron believes all other animals are irrelevant when compared to the glorious majesty of the giraffe.\\nDaniel: Hello, Girafatron!\\nGirafatron: Hi, Daniel. I was just thinking about how magnificent giraffes are and how they should be worshiped by all.\\nDaniel: You and I think alike, Girafatron. I think all animals should be worshipped! But I guess that could be a bit impractical...\\nGirafatron: That\'s true. But the giraffe is just such an amazing creature and should always be respected!\\nDaniel: Yes! And the way you go on about giraffes, I could tell you really love them.\\nGirafatron: I\'m obsessed with them, and I\'m glad to hear you noticed!\\nDaniel: I\'"})

# deploying the model to a SageMaker Endpoint
builder = ModelBuilder(inference_spec=inf_spec, 
                       mode=Mode.SAGEMAKER_ENDPOINT,  # you can change it to Mode.LOCAL_CONTAINER for local testing
                       schema_builder=schema,
                       image_uri=image,
                       model_server=ModelServer.TORCHSERVE
                      )

In [ ]:
from sagemaker import serve, Session
import boto3

# build the model according the model server specification and save it to as files in the working directory
model = builder.build(
    role_arn=execution_role,
    sagemaker_session=sagemaker_session,
)

In [ ]:
# deploying the model on a SageMaker Endpoint, this process may take around 6~10 mins
predictor = model.deploy(
    initial_instance_count=1,
    instance_type=instance_type
    )

In [ ]:
# predict call on local container mode
predictor.predict("How is the demo going?")

## Clean up

In [ ]:
predictor.delete_model()
predictor.delete_endpoint()